In [16]:
import math
import redis
import hashlib

class BloomFilter:
    # 初始化方法
    def __init__(self, n, p):
        # 初始化位数组长度 m
        self.m = self._calculate_m(n, p)
        # 预期存储的元素数量 n
        self.k = self._calculate_k(n, self.m)
        # 初始化存储位图的键
        self.redis_key = 'bloom_filter'
        # 初始化位数组
        self.redis = redis.Redis(host='localhost', port=6379, db=0)
        
    # 添加元素，对数据进行 k 次不同的哈希计算，并将第 k 位置1
    def add(self, item):
        for i in range(self.k):
            index = self._hash(item, i)
            self.redis.setbit(self.redis_key, index, 1)

    # 检查元素，判断元素是否一定不存在或者可能存在
    def contains(self, item):
        for i in range(self.k):
            index = self._hash(item, i)
            # 有一位为 0 则元素一定不存在，反之则可能存在
            if self.redis.getbit(self.redis_key, index) == 0:
                return False
        return True

    # 构造哈希函数，生成设置或检查索引
    def _hash(self, item, seed):
        # 实例化sha256对象
        h = hashlib.sha256()
        # 相当于对 item + seed 的组合计算哈希
        h.update(item.encode('utf-8'))
        h.update(str(seed).encode('utf-8'))
        # 返回 获取 SHA-256 的 32 字节哈希值，并将其高位在前转换成的大整数，对 m 取余将数据对应的每一位索引限制在位数组长度 m 中
        return int.from_bytes(h.digest(), byteorder='big') % self.m

    # 按照公式计算m（静态方法）
    @staticmethod
    def _calculate_m(n: int, p: float) -> int:
        m = -(n * math.log(p)) / (math.log(2) ** 2)
        return math.ceil(m)

    # 按照公式计算m
    @staticmethod
    def _calculate_k(n: int, m: int) -> int:
        k = (m / n) * math.log(2)
        return math.ceil(k)

if __name__ == '__main__':
    # 实例化布隆过滤器
    bf = BloomFilter(1000, 0.05)
    # 向空的布隆过滤器中添加元素
    for i in range(1000):
        bf.add('data' + str(i))
    # 
    true_count = 0
    for i in range(1000):
        if bf.contains('not_data' + str(i)):
            true_count += 1

    print(f"误判率: {(true_count / 1000)*100:.2f}%")

误判率: 5.20%
